In [1]:
import os
import random
import re
import requests
import time

In [2]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Subway Data

In [9]:
def get_coords(soup) -> Point:
    latitude_meta = soup.find("meta", {"itemprop": "latitude"})
    longitude_meta = soup.find("meta", {"itemprop": "longitude"})
    lat = float(latitude_meta.attrs["content"])
    lon = float(longitude_meta.attrs["content"])
    return Point(lon, lat)

In [10]:
scraper = cloudscraper.create_scraper()
url_root = "https://restaurants.subway.com"

In [11]:
count_dict = {}
store_list = []

for i in tqdm(range(len(states_df)), desc="Parsing States"):
    # Get State code
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"].lower()

    # Skip Territories
    if state_code in ["pr", "as", "vi", "gu", "mp"]:
        continue

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    state_url = f"{url_root}/united-states/{state_code}"
    state_r = scraper.get(state_url)

    if state_r.status_code != 200:
        print(states_df.iloc[i]["NAME"])
        print(state_url)
        continue

    soup = BeautifulSoup(state_r.text, "html.parser")
    locations_as = soup.find_all("a", {"class": "Directory-listLink"})

    # Parse all locations in a state
    for locations_a in tqdm(locations_as, desc=f"Parsing Locations In {state_code}"):
        href = locations_a.attrs["href"]
        count = int(re.findall(r"\d+", locations_a.attrs["data-count"])[0])
        location_url = href.replace("..", url_root)
        if count == 1:
            r_location = scraper.get(location_url)
            location_soup = BeautifulSoup(r_location.text, "html.parser")
            tbody = location_soup.find("tbody")
            schedule = list(
                set(
                    [
                        td.text
                        for td in tbody.find_all(
                            "td", {"class": "c-hours-details-row-intervals"}
                        )
                    ]
                )
            )
            if len(schedule) == 1 and schedule[0] == "Closed":
                print(location_url)
                continue
            point = get_coords(location_soup)
            store_dict = {
                "STATE": state_code,
                "geometry": point,
            }
            store_list.append(store_dict)
            state_store_list.append(store_dict)
        else:
            r_location = scraper.get(location_url)
            location_soup = BeautifulSoup(r_location.text, "html.parser")
            locations_divs = location_soup.find_all(
                "div", {"class": "Teaser-innerWrapper"}
            )

            # Multiple stores in locations, parse the stores per location
            for location_div in locations_divs:
                try:
                    location_a = location_div.find("a")
                    location_url = location_a.attrs["href"].replace("../..", url_root)
                    r_store = scraper.get(location_url)
                    store_soup = BeautifulSoup(r_store.text, "html.parser")
                    tbody = store_soup.find("tbody")
                    schedule = list(
                        set(
                            [
                                td.text
                                for td in tbody.find_all(
                                    "td", {"class": "c-hours-details-row-intervals"}
                                )
                            ]
                        )
                    )
                    if len(schedule) == 1 and schedule[0] == "Closed":
                        print(location_url)
                        continue
                    point = get_coords(store_soup)
                    store_dict = {
                        "STATE": state_code,
                        "geometry": point,
                    }
                    store_list.append(store_dict)
                    state_store_list.append(store_dict)
                except Exception as e:
                    print(e, location_url)
                time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        subway_state_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        subway_state_gdf.to_file(f"data/states/{state_code}.gpkg")

    time.sleep(random.uniform(0.01, 0.25))

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing Locations In ms:   0%|          | 0/142 [00:00<?, ?it/s]

https://restaurants.subway.com/united-states/ms/d'iberville/3821-promenade-parkway
https://restaurants.subway.com/united-states/ms/gautier/7204-martin-bluff-drive
https://restaurants.subway.com/united-states/ms/jackson/1263-high-street
https://restaurants.subway.com/united-states/ms/jackson/1400-john-r-lynch-st
https://restaurants.subway.com/united-states/ms/jackson/6351-i-55-n
https://restaurants.subway.com/united-states/ms/madison/175-grandview
https://restaurants.subway.com/united-states/ms/meridian/1729-hwy-39-n
https://restaurants.subway.com/united-states/ms/mississippi-state/85-old-main-street
https://restaurants.subway.com/united-states/ms/pass-christian/125-davis-avenue
https://restaurants.subway.com/united-states/ms/picayune/1701-highway-43-north
https://restaurants.subway.com/united-states/ms/port-gibson/1113-hwy-61-n
https://restaurants.subway.com/united-states/ms/woodville/187-us-highway-61


Parsing Locations In nc:   0%|          | 0/302 [00:00<?, ?it/s]

https://restaurants.subway.com/united-states/nc/asheboro/2003-north-fayetteville-street
https://restaurants.subway.com/united-states/nc/asheville/278-smokey-park-hwy
https://restaurants.subway.com/united-states/nc/asheville/4-old-eastwood-village-blvd
https://restaurants.subway.com/united-states/nc/black-mountain/202-nc-hwy-9
https://restaurants.subway.com/united-states/nc/boone/200-watauga-village-drive
https://restaurants.subway.com/united-states/nc/burlington/216-huffman-mill-rd
https://restaurants.subway.com/united-states/nc/burlington/2310-maple-avenue
https://restaurants.subway.com/united-states/nc/carthage/1005-monroe-street
https://restaurants.subway.com/united-states/nc/cary/611-e-chatham-street
https://restaurants.subway.com/united-states/nc/chapel-hill/11552-us-hwy-15-501-n
https://restaurants.subway.com/united-states/nc/charlotte/108-c-s-sharon-amity-road
https://restaurants.subway.com/united-states/nc/charlotte/1300-the-plaza
https://restaurants.subway.com/united-states/nc

...

https://restaurants.subway.com/united-states/in/albion/901-n-ridge-rd
https://restaurants.subway.com/united-states/in/clarksville/highway-131
https://restaurants.subway.com/united-states/in/edinburgh/gatling-streeet
https://restaurants.subway.com/united-states/in/fort-wayne/425-e-paulding-rd
https://restaurants.subway.com/united-states/in/fort-wayne/4935-bluffton-rd
https://restaurants.subway.com/united-states/in/highland/2305-main-street
https://restaurants.subway.com/united-states/in/indianapolis/3200-cold-spring
https://restaurants.subway.com/united-states/in/indianapolis/421-north-alabama
https://restaurants.subway.com/united-states/in/indianapolis/4607-s-harding-st
https://restaurants.subway.com/united-states/in/indianapolis/7137-n-michigan-rd
https://restaurants.subway.com/united-states/in/logansport/3424-e-market-street
https://restaurants.subway.com/united-states/in/mishawaka/2315-lincolnway-w
https://restaurants.subway.com/united-states/in/new-albany/3400-grantline-rd
https://

Parsing Locations In mn:   0%|          | 0/241 [00:00<?, ?it/s]

https://restaurants.subway.com/united-states/mn/arden-hills/3673-lexington-avenue-north
https://restaurants.subway.com/united-states/mn/blaine/4369-ball-rd
https://restaurants.subway.com/united-states/mn/brainerd/hwys-210-and-25
https://restaurants.subway.com/united-states/mn/burnsville/1100-county-rd-42
https://restaurants.subway.com/united-states/mn/burnsville/13700-13720-county-rd-11
https://restaurants.subway.com/united-states/mn/cook/221-s-hwy-53
https://restaurants.subway.com/united-states/mn/cottage-grove/7430-80th-st
https://restaurants.subway.com/united-states/mn/eden-prairie/9633-anderson-lakes-pkwy
https://restaurants.subway.com/united-states/mn/fosston/511-1st-st-west
https://restaurants.subway.com/united-states/mn/ham-lake/1444-147th-ave-ne
https://restaurants.subway.com/united-states/mn/hibbing/12080-hwy-169
https://restaurants.subway.com/united-states/mn/kenyon/701-2nd-st
https://restaurants.subway.com/united-states/mn/little-canada/3340-rice-st
https://restaurants.subwa

Parsing Locations In ct:   0%|          | 0/128 [00:00<?, ?it/s]

https://restaurants.subway.com/united-states/ct/bridgeport/244-university-avenue
https://restaurants.subway.com/united-states/ct/canaan/101-church-st
https://restaurants.subway.com/united-states/ct/danbury/73-mill-plain-rd
https://restaurants.subway.com/united-states/ct/deep-river/83-main-st
https://restaurants.subway.com/united-states/ct/east-hartford/1210-main-street
https://restaurants.subway.com/united-states/ct/enfield/323-hazard-avenue
https://restaurants.subway.com/united-states/ct/griswold/832-voluntown-rd
https://restaurants.subway.com/united-states/ct/groton/967-poquonnock-rd
https://restaurants.subway.com/united-states/ct/hamden/2411-whitney-avenue
https://restaurants.subway.com/united-states/ct/hebron/43-main-st
https://restaurants.subway.com/united-states/ct/manchester/288-center-st
https://restaurants.subway.com/united-states/ct/meriden/325-chamberlain-hwy
https://restaurants.subway.com/united-states/ct/new-britain/260-newington-ave
https://restaurants.subway.com/united-s

In [12]:
subway_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        subway_gdf = gpd.read_file(constructed_path)
        subway_gdfs.append(subway_gdf)

In [13]:
subway_gdf = gpd.GeoDataFrame(pd.concat(subway_gdfs, ignore_index=True))
subway_gdf = subway_gdf.to_crs(9311)
subway_gdf.to_file(f"data/stores.gpkg")

In [14]:
subway_by_state_series = subway_gdf.groupby(["STATE"]).size()

In [15]:
subways_by_state_df = pd.DataFrame(
    {
        "STUSPS": subway_by_state_series.index,
        "COUNT": subway_by_state_series.values,
    }
)

In [16]:
subways_by_state_df["STUSPS"] = subways_by_state_df["STUSPS"].str.upper()

In [17]:
subways_by_states_gdf = states_with_population_df.merge(
    subways_by_state_df, on="STUSPS", how="inner"
)

In [18]:
subways_by_states_gdf["per_1000"] = subways_by_states_gdf["COUNT"] / (
    subways_by_states_gdf["POPULATION"] / 1000
)
subways_by_states_gdf["per_10k"] = subways_by_states_gdf["COUNT"] / (
    subways_by_states_gdf["POPULATION"] / 10_000
)
subways_by_states_gdf["per_100k"] = (
    (subways_by_states_gdf["COUNT"] / (subways_by_states_gdf["POPULATION"] / 100000))
    .round(decimals=0)
    .astype(int)
)
subways_by_states_gdf["per_500k"] = subways_by_states_gdf["COUNT"] / (
    subways_by_states_gdf["POPULATION"] / 500_000
)
subways_by_states_gdf["per_1m"] = (
    (subways_by_states_gdf["COUNT"] / (subways_by_states_gdf["POPULATION"] / 1_000_000))
    .round(decimals=0)
    .astype(int)
)

In [19]:
subways_by_states_gdf = subways_by_states_gdf.dropna()

In [21]:
subways_by_states_gdf = subways_by_states_gdf.to_crs(9311)
subways_by_states_gdf.to_file(f"data/stores_by_states.gpkg")